In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_labels = pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")

In [ ]:
df_labels

In [ ]:
df_labels['id'] = df_labels['id'] + '.jpg'

In [ ]:
df_labels

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_validation = train_test_split(df_labels, test_size=0.2, shuffle=True)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
        rescale=1./255)

training_set = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory='/kaggle/input/dog-breed-identification/train',
    x_col="id",
    y_col="breed",
    target_size=(150, 150),
    class_mode="categorical",
    batch_size=128
)

validation_set = validation_datagen.flow_from_dataframe(
    dataframe=df_validation,
    directory='/kaggle/input/dog-breed-identification/train',
    x_col="id",
    y_col="breed",
    target_size=(150, 150),
    class_mode="categorical",
    batch_size=128
)

In [ ]:
classes = df_labels['breed'].unique().shape[0]
classes

In [ ]:
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout       

from keras.models import Sequential


In [ ]:
model=Sequential()


model.add(Conv2D(32, (3, 3),input_shape=(150,150,3)))   #32 filters, each filter 3x3
model.add(Activation('relu'))


model.add(MaxPooling2D(pool_size=(2,2)))
#reduces dimensions of image- (12,12,64)
#4 pixelels merged into 1, channels remains the same

model.add(Conv2D(32, (3, 3)) )  #16 filters, each filter 3x3
model.add(Activation('relu'))


model.add(Flatten())   #After flattening.. total features will be 28*28=784

model.add(Dropout(0.25))  #25% of random weights will be reduced to zero

model.add(Dense(units=128, activation='relu'))

# Step 5 - Output Layer
model.add(Dense(units=classes, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(x = training_set, validation_data = validation_set, epochs = 20)

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1/255. 
)

test_set = test_datagen.flow_from_directory (
    '/kaggle/input/dog-breed-identification',
    target_size = (150,150),
    batch_size = 128,
    classes=['test']
)

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
df_submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
df_submission

In [ ]:
import re
file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)

In [ ]:
id_list[:10]

In [ ]:
df_submission['id'] = id_list
df_submission.iloc[:,1:] = y_pred
df_submission

In [ ]:
final_df = df_submission.set_index('id')
final_df.to_csv('Submission.csv')